# Functions and classes for the project

## For before and now: a class to store, for an image at a given rescale, the slifing wondows data: dataset, positions and scores

In [133]:
class FramesAtGivenScaledImage():
    """Represents the images chunks for the scaled image at one size. 
    Instances are made before the NN feeding to store the datasets and the positions of the frames/image chunks, 
    and also used after the NN feeding to store the scores"""
    
    def __init__(self, scaling_factor, dataset, positions):
        self.scaling_factor = scaling_factor  # The factor used to scale/reduce the image
        self.dataset = dataset  # Dataset of image frames => this is a CustomDatasetFromImages instance
        self.positions = positions  # Position tuples (x,y) associated to the frames. Positions are from top left.
        self.scores = []  # Scores associated to the frames. These are scalars from -2 to 2. Filled with NN output, method set_scores
        
    def set_scores(net):
        """net: Neural network used to compute the scores, given the dataset TODO"""
        pass
    
    
def get_center(frame_position):
    """Returns the center associated to frame_position in the scaled image 
    (so the size of a frame is not yet resized:36x36)
    """
    return (frame_position[0]+18, frame_position[1]+18)
        

## Classes and methods for getting subdetections from frames 

In [136]:
class DetectionCandidate:
    """It represents a square that represents a detection in the image rescaled to
    normal size. Let we call it a subdetection"""
    def __init__(self, score, position, dims):
        """dimensions of each detection (in the resized image). For example, if the sliding window
        was sliding an image that has been reduced by /1.2, the dimension of each subdetection will
        be (1.2*36, 1.2*36)"""
        self.score = score   # Score associated to the image frame the subdetection comes from
        self.position = position  # Position associated to the image frame the subdetection comes from, BUT in the rescaled image
        self.dims = dims # Dimension tuple (wifth, height) of the image frame the subdetection comes from, BUT in the rescaled image
        
    def get_center(self):
        return (self.position[0]+self.dims[0]/2, self.position[1]+self.dims[1]/2)
    
    def computer_center_dists(self, other_square): # TODO: useless???
        """Compute norm 2 between actual square center and another square center"""
        return sqrt((self.position[0] - other_square.position[0])
                   *(self.position[0] - other_square.position[0])
                   +(self.position[1] - other_square.position[1])
                   *(self.position[1] - other_square.position[1]))
    
    def __str__(self):
        return "score:" + str(self.score) + "; position:" + str(self.position) + "; dims:" + str(self.dims)
        

def capture_good_positions(framesAtGivenScaledImages): # TODO: seems that the scaling of the positions is false
    """From the list of framesAtGivenScaledImages, we build a list of DetectionCandidates. These are derived from
    the chunks whose associated score is =>0.
    Returns a list of DetectionCandidates instance 
    """
    detectionCandidates = []
    for fagsi in framesAtGivenScaledImages:
        print("fagsi.scores:", fagsi.scores)
        for i in range(len(fagsi.scores)):
            if fagsi.scores[i] >= 0:
                newdims = (int(36*fagsi.scaling_factor), int(36*fagsi.scaling_factor))
                print("newdims:", newdims)
                """To get the position in the rescaled image (recall that it is top-left), 
                we first scale linearly the center, and then from the center, we deduced the new position
                """
                old_center = get_center(fagsi.positions[i]) # center in the scaled image (so before rescaling to original size)
                newx = old_center[0]*fagsi.scaling_factor
                newx = int(newx - newdims[0]/2)
                newy = old_center[1]*fagsi.scaling_factor
                newy = int(newy - newdims[1]/2)
                """detectionCandidates.append(DetectionCandidate(fagsi.scores[i], 
                                                              (int(fagsi.positions[i][0]*fagsi.scaling_factor),
                                                               int(fagsi.positions[i][1]*fagsi.scaling_factor)),
                                                               (newdims))"""
                detectionCandidates.append(DetectionCandidate(fagsi.scores[i], (newx, newy), newdims))
                                            
    return detectionCandidates                                      


## Functions for getting detections from subdetections AND filtering them at the same time
Should be in a file concerning the clustering/detections

In [114]:
import numpy as np
from sklearn.cluster import DBSCAN

def cluster_frames(centers_frames, eps=3, min_samples=1):
    """
    :param eps: maximum distance between two samples
    :param min_samples: number of samples in a neighborhood for a point to be a core point
    :param centers_frames: List of coordinates of the center of frames (x,y)
    :return: A list of lists --> for one list, there is the indexes of the frames
    We don't have an explicit class to design a Cluster, which is a list of (subdetection) indices.
    """
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(centers_frames)
    dict_clusters = dict()
    for index, value in enumerate(clustering.labels_):
        # If -1, it means the subdetection does not have enough neighbor => we ignore it and don't build a cluster from it.
        if value != -1:
            if value not in dict_clusters:
                dict_clusters[value] = [index]
            else:
                dict_clusters[value].append(index)
    return list(dict_clusters.values())


def getDetections(subdetections, min_samples=1):
    """Returns a list of Detections.
    Detections are clusters. A Detection is a list of indices of subdetections that represent it.
    min_samples: number of minimum subdetections in the detection for the detection to be accepted. Otherwise, 
    the subdetection is ignored
    => With this function, we both do the steps of getting the detections and filtering them (discarding those with
    too few subdetections)!
    """
    centers_frames = []
    for subd in subdetections:
        center = subd.get_center()
        centers_frames.append([center[0], center[1]]) # because DBSCAN works with vectors that are lists, not tuples
    clusters = cluster_frames(centers_frames, 50, min_samples) # TODO: eps must be proportional to the image dims
    return clusters

## Function for keeping the best subdetection for each detection

In [115]:
"""def get_best_cluster_candidate(subdetections, cluster):
    
    subdetections: a list of DetectionCandidates
    cluster: a list of subdetection indices
    Returns the chosen candidate for given cluster
    best_score = -1000
    best_candidate_index = -1000
    for ind, candidate in enumerate(cluster):
        if best_score < candidate.score:
            best_score = candidate.score
            best_candidate_index = ind
    return cluster[best_candidate_index]"""


def get_best_clusters_candidates(subdetections, clusters):
    """
    subdetections: a list of DetectionCandidates
    clusters: a list of clusters
    Returns the chosen candidate for each cluster 
    => we have a list of DetectionCandidates. Each one represents the Detection that contains it."""
    chosen = []
    best_score = -1000
    best_candidate_index = -1000
    for c in clusters:
        best_score = -1000
        best_candidate_index = -1000
        for ind in c: # indices of our list of subdetections
            if best_score < subdetections[ind].score:
                best_score = subdetections[ind].score
                best_candidate_index = ind
        chosen.append(subdetections[best_candidate_index])
    return chosen
        

## Class for saving the detections on an image

In [116]:
# We first have a function to have a random color
import random

def random_color():
    return (random.randint(0,255), random.randint(0,255), random.randint(0,255))

In [117]:
from PIL import Image, ImageDraw

class ImageWithDetections:
    
    def __init__(self, im, subdetections):
        self.im = im
        self.subdetections = subdetections
        
    def save(self, filename="saved_detections.JPG"):
        """Save the image as well as the kept DetectionCanditates."""
        im = self.im.copy()
        draw = ImageDraw.Draw(im)
        fntsize = 18
        fnt = ImageFont.truetype("impact.ttf",fntsize)
        for subd in self.subdetections:
            randomc = random_color()
            draw.rectangle((subd.position[0],subd.position[1],
                            subd.position[0]+subd.dims[0],subd.position[1]+subd.dims[1]), 
                           outline=randomc) # But we can't specify border width :(
            draw.text((subd.position[0],subd.position[1]-fntsize), str(subd.score), fill=randomc, font=fnt)
        im.save(filename, "JPEG")
        
        

# Plaing around around

## Drawing a rectangle and writing a text on an image.

In [11]:
# %matplotlib

In [94]:
from PIL import Image, ImageDraw, ImageFont

pilImage = Image.open("catch_detec_images/IMGP0017.JPG")
draw = ImageDraw.Draw(pilImage) # type: ImageDraw. Its existing affects the image pilImage.
# draw.rectangle((100,200, 500,300), outline="red") 
draw.rectangle((100,200, 500,300), outline=(255,0,0)) # But we can't specify border width :(

# text with font size of 100 px.
fnt = ImageFont.truetype("impact.ttf",100)
draw.text((10,10), "Hello World", fill=(255,255,0), font=fnt)


pilImage.save("catch_detec_images/withdrawing" + ".JPG", "JPEG")

# Testing whole pipeline, from the lists of scores and positions at diff scalings (so from FramesAtGivenScaledImage instances) the to the image with detections

## Let we have these FramesAtGivenScaledImage instances, so we have the frames at different scales with positions and refined scores associated:

In [134]:
# For each image size, we won't add all the possible frames, it will be too long. But it's not important to test.
# We don't need the dataset to test, this was for the NN step

# Will be done before NN feeding
fagsi1 = FramesAtGivenScaledImage(1, [], [(0,0), (95,130), (100,130), (500,300)])  
fagsi2 = FramesAtGivenScaledImage(1.2, [], [(0,0), (81,105), (333,458), (500,300)])  
fagsi3 = FramesAtGivenScaledImage(2.4, [], [(0,1), (38,53), (167,230), (312,33), (400,20)]) 

# Will be done after NN feeding. TODO; implement set_scores
fagsi1.scores = [-1, 0.6, 0.65, -0.4]
fagsi2.scores = [-1, 0.7, 0.7, -0.4]
fagsi3.scores = [-1.2, 0.53, 0.8, 0.9, -0.1]


framesAtGivenScaledImages = []
framesAtGivenScaledImages.append(fagsi1)
framesAtGivenScaledImages.append(fagsi2)
framesAtGivenScaledImages.append(fagsi3)

## Frome these, we get the subdetections.

In [137]:
subdetections = capture_good_positions(framesAtGivenScaledImages)
print("subdetections:")
for subd in subdetections:
    print(subd)
# OK!

fagsi.scores: [-1, 0.6, 0.65, -0.4]
newdims: (36, 36)
newdims: (36, 36)
fagsi.scores: [-1, 0.7, 0.7, -0.4]
newdims: (43, 43)
newdims: (43, 43)
fagsi.scores: [-1.2, 0.53, 0.8, 0.9, -0.1]
newdims: (86, 86)
newdims: (86, 86)
newdims: (86, 86)
subdetections:
score:0.6; position:(95, 130); dims:(36, 36)
score:0.65; position:(100, 130); dims:(36, 36)
score:0.7; position:(97, 126); dims:(43, 43)
score:0.7; position:(399, 549); dims:(43, 43)
score:0.53; position:(91, 127); dims:(86, 86)
score:0.8; position:(401, 552); dims:(86, 86)
score:0.9; position:(749, 79); dims:(86, 86)


## Clustering of DetectionCandidates into Detections and filtering 

In [138]:
detections = getDetections(subdetections,min_samples=2)  # OK!
print(detections) # In the example, one subdetection is alon in a detection => this detection is discarded.

[[0, 1, 2, 4], [3, 5]]


## Get the best candidates

In [139]:
winners = get_best_clusters_candidates(subdetections, detections)
for w in winners:
    print(w)

score:0.7; position:(97, 126); dims:(43, 43)
score:0.8; position:(401, 552); dims:(86, 86)


## Save an image with all subdetections, and then only with the kept subdetections

In [140]:
from PIL import Image, ImageDraw

# With all subdetections
pilImage = Image.open("catch_detec_images/blank_example.jpg")
imdet = ImageWithDetections(pilImage, subdetections)
imdet.save("catch_detec_images/all_detections.JPG")

# With only winner subdetections
imdet.subdetections = winners
imdet.save("catch_detec_images/winner_detections.JPG")